In [1]:
#import os
#from osgeo import gdal
import numpy as np
import math
import json
import copy
#  jr,  judging whether the grid is in the river，is = 1, not = 0
# flow_acc, the raster(converted to ASCII code) that shows the accumulated flow to each cell.


In [2]:
def write_dict(variable,name):
    ''' input: variable to write
       inout: name, string file name
        output: a txt named name.txt
    '''
    import json
    js = json.dumps(variable)   
    file = open('ASCII/{}.txt'.format(name), 'w')  
    file.write(js)  
    file.close()  
#write_dict(all_paths_0,'all_paths_0')
def read_dict(name):
    ''' input: a txt name, filename to read
    '''
    file = open('ASCII/{}.txt'.format(name), 'r') 
    js = file.read()
    dic = json.loads(js)   
    return (dic)
    file.close() 
# file=open('ASCII/all_paths.txt','w') 
# file.write(str(all_paths)); 
# file.close() 

### 1 读取数据

In [6]:
all_paths = read_dict('all_paths_0')
river_s = all_paths['river_s']
bound_s = all_paths['bound_s']

In [19]:
fd = np.loadtxt('ASCII/fd.txt',skiprows = 6)
am4 = np.loadtxt('ASCII/am4.txt',skiprows = 6)
b = np.loadtxt('ASCII/b.txt',skiprows = 6)
jr = np.loadtxt('ASCII/jr.txt',skiprows = 6)

In [8]:
river_s[-1]

[[[558], [336]], [[557], [337]]]

In [9]:
jr[557,337]

1.0

### 2 初始化值

In [33]:
def del_99(arr_tif0,bounds,null_d):
    '''初始化空值为-9999
    '''
    arr_tif = copy.deepcopy(arr_tif0)
    for i in bounds: 
        for j in i: 
            arr_tif[j] = [null_d]
    return arr_tif
#del_99(fd,bound_s,0)
def ini_tif(arr_tif0,d,jr,bounds,null_d):
    '''初始化值为d
    '''
    arr_tif  = copy.deepcopy(arr_tif0) 
    [rows, cols] = arr_tif.shape 
    for i in range(rows - 1): 
        for j in range(cols-1): 
            if arr_tif[i, j] != -9999:
                if jr[i, j] == 1:
                    arr_tif[i, j] = null_d
                else:
                    arr_tif[i, j] = d
    arr_tif1 = copy.deepcopy(arr_tif) 
    arr_tif2 = del_99(arr_tif1,bound_s,null_d)
    return arr_tif2

k_arr = ini_tif(fd,0,jr,bound_s,-9999)
reg_arr = ini_tif(fd,0,jr,bound_s,-9999)

In [39]:
def rows_cols(ind, path_one):
    '''对于位置ind，找到所有下游的位置坐标
    '''
    rows = []
    cols = []
    if ind < len(path_one) - 2:
        while ind < len(path_one) - 2:
            rows += path_one[ind+1][0]
            cols += path_one[ind+1][1]
            ind +=1
        return [rows,cols]
    else:
        print ('ind out of index of path_one')
        
def k_all(k_arr0,all_paths,reg_arr0):
    k_arr = copy.deepcopy(k_arr0)
    reg_arr = copy.deepcopy(reg_arr0)
    for path in all_paths:
#         print(path)
#         print(len(path)-2)
        for ind in range(0,len(path)-2):
            ind_0 = path[ind]
            if reg_arr[ind_0][0] ==0:
                row_col = rows_cols(ind, path)
                k_arr[row_col ] += am4[ind_0][0]/(1-b[row_col ])
                reg_arr[ind_0] = 1
            else:
                break
    return k_arr

In [40]:
k_arr = k_all(k_arr,river_s,reg_arr)

In [41]:
np.savetxt('ASCII/k_arr.txt',k_arr,fmt='%0.6f') # 没有前6行

In [42]:
def add_begin(name):
    with open('ASCII/begin.txt', 'r+') as f:
        content = f.read() 
        with open('ASCII/{}.txt'.format(name), 'r+') as f:
            content0 = f.read() 
            f.seek(0, 0)
            f.write(content + '\n' + content0)
            f.close()
        f.close()

In [43]:
add_begin('k_arr')

In [ ]:
def pow_all2(pow_arr0,p,slope,all_paths,reg_arr0):
    pow_arr = copy.deepcopy(pow_arr0)
    reg_arr = copy.deepcopy(reg_arr0)
    for path in all_paths:
#         print(path)
#         print(len(path)-2)
        for ind in range(0,len(path)-2):
            ind_0 = path[ind]
            if reg_arr[ind_0][0] ==0:
                pl_s = pl_slope2(p,ind, path,slope)
               # print(pl_s)
                row_col = rows_cols(ind, path)
               # print(row_col)
                pow_arr[row_col] += pl_s 
                #print(pow_arr[row_col])
                reg_arr[ind_0] = 1
            else:
                break
    return pow_arr

## 1 读取txt到数组

In [9]:
b = np.loadtxt('ASCII/b.txt',skiprows = 6)
p = np.loadtxt('ASCII/p.txt',skiprows = 6)
fd = np.loadtxt('ASCII/fd.txt',skiprows = 6)
jr = np.loadtxt('ASCII/jr.txt',skiprows = 6)
slope = np.loadtxt('ASCII/slope.txt',skiprows = 6)
flow_acc = np.loadtxt('ASCII/flow_acc.txt',skiprows = 6)
am4 = np.loadtxt('ASCII/am4.txt',skiprows = 6)

In [3]:
flow_acc.shape 

(569, 538)

## 1.1 例子遍历数组

## 2 得到起点索引列表

In [4]:
def origin_list(flow_acc, jr):
    '''
   output: A list of all starting points for all flow paths
    '''
    list_0 = []
    [rows, cols] = flow_acc.shape 
    for i in range(rows - 1): 
        for j in range(cols-1): 
            if flow_acc[i,j] == 0 and jr[i, j] == 0: #  jr[i, j] == 0 起点处于非河道
                 list_0.append([[i],[j]])
    return list_0

In [5]:
ori_list = origin_list(flow_acc,jr) # A list of all starting points for all flow paths

In [6]:
ori_list[0]

[[5], [84]]

In [7]:
fd[ori_list[0]]

array([8.])

In [8]:
len(ori_list)

59914

## 3 起点路径列表

In [143]:
def flow_path(ori, jr,fd):
    '''
   output:  lists of all path points for all flow paths
    '''
    list1 = []
    row = ori[0][0]
    col = ori[1][0]
    while jr[row,col] != 1 and fd[row,col] != -9999:
        if [row,col] not in list1:
            list1.append([[row],[col]])
            if fd[row,col] == 1:
                col += 1
            elif fd[row,col] == 2:
                row += 1
                col += 1
            elif fd[row,col] == 4:
                row += 1
            elif fd[row,col] == 8:
                row += 1
                col -= 1
            elif fd[row,col] == 16:
                col -= 1
            elif fd[row,col] == 32:
                row -= 1
                col -= 1
            elif fd[row,col] == 64:
                row -= 1
            elif fd[row,col] == 128:
                row -= 1
                col += 1
    list1.append([[row],[col]]) # 包含受内水体和流域出口
    return list1

def river_paths(ori_list, jr,fd):
    '''
   output:  lists of all path points for all flow paths
    '''
    list0 = []
    for ori in ori_list:
        list1 = flow_path(ori, jr,fd)
        if jr[list1[-1]][0] == 1:
            list0.append(list1)
    return list0
def bound_paths(ori_list, jr,fd):
    '''
   output:  lists of all path points for all flow paths
    '''
    list0 = []
    for ori in ori_list:
        list1 = flow_path(ori, jr,fd)
        if jr[list1[-1]][0] == -9999:
            list0.append(list1)
    return list0

In [102]:
river_s = river_paths(ori_list, jr,fd)

In [103]:
bound_s = bound_paths(ori_list, jr,fd)

In [104]:
len(river_s)

53172

In [105]:
len(bound_s)

6642

In [107]:
river_s[-4]

[[[556], [335]], [[555], [336]], [[555], [337]]]

In [109]:
jr[river_s[-4][-1]][0]

1.0

In [110]:
all_paths_0 = {'river_s': river_s, 'bound_s': bound_s}

In [7]:
def write_dict(variable,name):
    ''' input: variable to write
       inout: name, string file name
        output: a txt named name.txt
    '''
    import json
    js = json.dumps(variable)   
    file = open('ASCII/{}.txt'.format(name), 'w')  
    file.write(js)  
    file.close()  
#write_dict(all_paths_0,'all_paths_0')
def read_dict(name):
    ''' input: a txt name, filename to read
    '''
    file = open('ASCII/{}.txt'.format(name), 'r') 
    js = file.read()
    dic = json.loads(js)   
    return (dic)
    file.close() 
# file=open('ASCII/all_paths.txt','w') 
# file.write(str(all_paths)); 
# file.close() 

In [112]:
write_dict(all_paths_0,'all_paths_0')

In [8]:
all_paths = read_dict('all_paths_0')

In [9]:
river_s = all_paths['river_s']

In [10]:
bound_s = all_paths['bound_s']

In [116]:
river_s[-4]

[[[556], [335]], [[555], [336]], [[555], [337]]]

## 4 计算动力作用
(1) 初始化计算栅格非空值为1

(2) 计算1+坡度的3次幂

(3) 计算L

(4) 计算pl

(5) 计算最终结果

In [150]:
def del_99(arr_tif0,bounds,null_d):
    '''初始化空值为-9999
    '''
    arr_tif = copy.deepcopy(arr_tif0)
    for i in bounds: 
        for j in i: 
            arr_tif[j] = [null_d]
    return arr_tif
#del_99(fd,bound_s,0)
def ini_tif(arr_tif0,d,jr,bounds,null_d):
    '''初始化值为d
    '''
    arr_tif  = copy.deepcopy(arr_tif0) 
    [rows, cols] = arr_tif.shape 
    for i in range(rows - 1): 
        for j in range(cols-1): 
            if arr_tif[i, j] != -9999:
                if jr[i, j] == 1:
                    arr_tif[i, j] = null_d
                else:
                    arr_tif[i, j] = d
    arr_tif1 = copy.deepcopy(arr_tif) 
    arr_tif2 = del_99(arr_tif1,bound_s,null_d)
    return arr_tif2

pow_arr = ini_tif(fd,1,jr,bound_s,-9999)
reg_arr = ini_tif(fd,0,jr,bound_s,-9999)

In [208]:
river_s[-4]

[[[556], [335]], [[555], [336]], [[555], [337]]]

In [153]:
pow_arr[555,336]

1.0

In [154]:
reg_arr[555,336]

0.0

In [155]:
def distance_list(ind, path_one):
    '''input: ind,某河流路径的某索引位置
      input: path_0, 某河流路径的位置列表
      output: ind下游的所有距离
    '''
    list0 = []
    if ind < len(path_one) - 2:
        d = 0
        while ind < len(path_one) - 2:
            if abs(path_one[ind][0][0]- path_one[ind+1][0][0]) + abs(path_one[ind][1][0]- path_one[ind+1][1][0]) == 2:
                d += math.sqrt(2)
            else:
                d += 1
            list0.append(d)
            ind +=1
        return list0
    else:
        print ('ind out of index of path_one')
#distance_list(1, path_one)

def rows_cols(ind, path_one):
    rows = []
    cols = []
    if ind < len(path_one) - 2:
        while ind < len(path_one) - 2:
            rows += path_one[ind+1][0]
            cols += path_one[ind+1][1]
            ind +=1
        return [rows,cols]
    else:
        print ('ind out of index of path_one')
        
def pl_list(p,ind, path_one):
    list0 = distance_list(ind, path_one)
    list1 = [pow(p,i) for i in list0]
    return list1
def pl_slope(p,ind, path_one,slope):
    row = path_one[ind][0][0]
    col = path_one[ind][1][0]
    theta = slope[row,col]
    list0 = distance_list(ind, path_one)
    list1 = [pow(p,i)*(1+theta**0.3) for i in list0]
    return list1
def pl_slope2(p,ind, path_one,slope):
    row = path_one[ind][0][0]
    col = path_one[ind][1][0]
    theta = slope[row,col]
    list0 = distance_list(ind, path_one)
    list1 = [pow(p,i)*(1+0.1*theta**0.3) for i in list0]
    return list1
###[[[565], [297]], [[565], [296]], [[566], [296]]]

def pow_all1(pow_arr0,p,slope,all_paths,reg_arr0):
    pow_arr = copy.deepcopy(pow_arr0)
    reg_arr = copy.deepcopy(reg_arr0)
    for path in all_paths:
#         print(path)
#         print(len(path)-2)
        for ind in range(0,len(path)-2):
            ind_0 = path[ind]
            if reg_arr[ind_0][0] ==0:
                pl_s = pl_slope(p,ind, path,slope)
               # print(pl_s)
                row_col = rows_cols(ind, path)
               # print(row_col)
                pow_arr[row_col] += pl_s 
                #print(pow_arr[row_col])
                reg_arr[ind_0] = 1
            else:
                break
    return pow_arr
#pow_result = pow_all(pow_arr,p,slope,all_paths)
def pow_all2(pow_arr0,p,slope,all_paths,reg_arr0):
    pow_arr = copy.deepcopy(pow_arr0)
    reg_arr = copy.deepcopy(reg_arr0)
    for path in all_paths:
#         print(path)
#         print(len(path)-2)
        for ind in range(0,len(path)-2):
            ind_0 = path[ind]
            if reg_arr[ind_0][0] ==0:
                pl_s = pl_slope2(p,ind, path,slope)
               # print(pl_s)
                row_col = rows_cols(ind, path)
               # print(row_col)
                pow_arr[row_col] += pl_s 
                #print(pow_arr[row_col])
                reg_arr[ind_0] = 1
            else:
                break
    return pow_arr

In [156]:
pow_result1 = pow_all1(pow_arr,0.5,slope,river_s,reg_arr) # 约3-8min

In [157]:
pow_result2 = pow_all2(pow_arr,0.5,slope,river_s,reg_arr) # 约3-8min

In [158]:
np.savetxt('ASCII/pow_all_1.txt',pow_result1,fmt='%0.6f') # 没有前6行

In [159]:
np.savetxt('ASCII/pow_all_2.txt',pow_result2,fmt='%0.6f') # 没有前6行

In [160]:
def add_begin(name):
    with open('ASCII/begin.txt', 'r+') as f:
        content = f.read() 
        with open('ASCII/{}.txt'.format(name), 'r+') as f:
            content0 = f.read() 
            f.seek(0, 0)
            f.write(content + '\n' + content0)
            f.close()
        f.close()

In [161]:
add_begin('pow_all_1')

In [162]:
add_begin('pow_all_2')

In [163]:
pow_result1[555,336]

3.9340186932192456

In [164]:
pow_result2[555,336]

2.306094673843758

### test1 数学公式测试

In [2]:
pow(2,3)

8

In [48]:
2**3

8

In [25]:
math.sqrt(2)

1.4142135623730951

In [22]:
abs(-2)

2

### test2 河流下游距离计算

In [20]:
path_one = all_paths[0] # [[5, 83], [6, 82], [7, 81], [7, 80]]
all_paths[0]

[[[5], [83]], [[6], [82]], [[7], [81]], [[7], [80]]]

In [19]:
def distance_list(ind, path_one):
    '''input: ind,某河流路径的某索引位置
      input: path_0, 某河流路径的位置列表
      output: ind下游的所有距离
    '''
    list0 = []
    if ind < len(path_one) - 2:
        d = 0
        while ind < len(path_one) - 2:
            if abs(path_one[ind][0][0]- path_one[ind+1][0][0]) + abs(path_one[ind][1][0]- path_one[ind+1][1][0]) == 2:
                d += math.sqrt(2)
            else:
                d += 1
            list0.append(d)
            ind +=1
        return list0
    else:
        print ('ind out of index of path_one')
#distance_list(1, path_one)

def rows_cols(ind, path_one):
    rows = []
    cols = []
    if ind < len(path_one) - 2:
        while ind < len(path_one) - 2:
            rows += path_one[ind+1][0]
            cols += path_one[ind+1][1]
            ind +=1
        return [rows,cols]
    else:
        print ('ind out of index of path_one')
        
def pl_list(p,ind, path_one):
    list0 = distance_list(ind, path_one)
    list1 = [pow(p,i) for i in list0]
    return list1
def pl_slope(p,ind, path_one,slope):
    row = path_one[ind][0][0]
    col = path_one[ind][1][0]
    theta = slope[row,col]
    list0 = distance_list(ind, path_one)
    list1 = [pow(p,i)*(1+theta**0.3) for i in list0]
    return list1

###[[[565], [297]], [[565], [296]], [[566], [296]]]

def pow_all(pow_arr0,p,slope,all_paths):
    pow_arr = copy.deepcopy(pow_arr0)
    for path in all_paths:
#         print(path)
#         print(len(path)-2)
        for ind in range(0,len(path)-2):
            pl_s = pl_slope(p,ind, path,slope)
           # print(pl_s)
            row_col = rows_cols(ind, path)
           # print(row_col)
            pow_arr[row_col] += pl_s 
            #print(pow_arr[row_col])
    return pow_arr
#pow_result = pow_all(pow_arr,p,slope,all_paths)

In [28]:
all_paths[0:2]

[[[[5], [83]], [[6], [82]], [[7], [81]], [[7], [80]]],
 [[[5], [84]], [[6], [83]], [[7], [82]], [[7], [81]], [[7], [80]]]]

In [22]:
distance_list(0, path_one)

[1.4142135623730951, 2.8284271247461903]

In [23]:
rows_cols(0, path_one)

[[6, 7], [82, 81]]

In [24]:
pl_list(0.5,0, path_one)

[0.37521422724648174, 0.14078571632817444]

In [25]:
pl_slope(0.5,0, path_one,slope)

[1.1455293720580795, 0.42981891812491974]

### test3 array 赋值

In [132]:
arr = np.arange(63).reshape((7,9))
print(arr)
#ss = [[1,0], [5, 3]]
#print(arr[ss])
print(arr[[1, 5, 6, 2], [0, 3, 1, 2]])
#arr[[1, 5, 6, 2], [0, 3, 1, 2]] += [0,1,2,3]
#arr[[1, 5, 6, 2], [0, 3, 1, 2]] *= [0,1,2,3]
#print(arr[[1,0], [5, 3]])
print(arr[[1,0], [5, 3]].prod())
print(arr[[1, 5, 6, 2], [0, 3, 1, 2]]) # 打印arr[1, 0]、arr[5, 3]，arr[7, 1]和arr[2, 2]

[[ 0  1  2  3  4  5  6  7  8]
 [ 9 10 11 12 13 14 15 16 17]
 [18 19 20 21 22 23 24 25 26]
 [27 28 29 30 31 32 33 34 35]
 [36 37 38 39 40 41 42 43 44]
 [45 46 47 48 49 50 51 52 53]
 [54 55 56 57 58 59 60 61 62]]
[ 9 48 55 20]
42
[ 9 48 55 20]


In [12]:
arr = np.arange(12).reshape((3,4))
print(arr)

[[ 0  1  2  3]
 [ 4  5  6  7]
 [ 8  9 10 11]]


In [17]:
print(arr[[1, 2], [0, 3]])
print(1/(arr[[1, 2], [0, 3]]*(-1)+1))

[ 4 11]
[-0.33333333 -0.1       ]


## 5 计算阻力作用

### (1) 初始化判断栅格

In [182]:
def del_99(arr_tif0,bounds,null_d):
    '''初始化空值为-9999
    '''
    arr_tif = copy.deepcopy(arr_tif0)
    for i in bounds: 
        for j in i: 
            arr_tif[j] = [null_d]
    return arr_tif
#del_99(fd,bound_s,0)
def ini_tif(arr_tif0,d,jr,bounds,null_d):
    '''初始化值为d
    '''
    arr_tif  = copy.deepcopy(arr_tif0) 
    [rows, cols] = arr_tif.shape 
    for i in range(rows - 1): 
        for j in range(cols-1): 
            if arr_tif[i, j] != -9999:
                if jr[i, j] == 1:
                    arr_tif[i, j] = null_d
                else:
                    arr_tif[i, j] = d
    arr_tif1 = copy.deepcopy(arr_tif) 
    arr_tif2 = del_99(arr_tif1,bound_s,null_d)
    return arr_tif2

reg_arr = ini_tif(fd,0,jr,bound_s,-9999) # if value == 0,需要计算
bv_arr_1 = ini_tif(fd,1,jr,bound_s,-9999)
bv_arr = ini_tif(fd,1,jr,bound_s,-9999)

In [168]:
bv_arr[555,336]

1.0

In [169]:
reg_arr[555,336]

0.0

### (2) 计算每个栅格的阻力

In [183]:
def b_theta1(arr_tif0, b, slope):
    arr_tif = copy.deepcopy(arr_tif0)
    [rows, cols] = arr_tif.shape 
    for i in range(rows - 1): 
        for j in range(cols-1): 
            if arr_tif[i, j] != -9999:
                arr_tif[i, j] = (1-b[i, j])*(1+slope[i, j]**0.3)
    return arr_tif
def b_theta2(arr_tif0, b, slope):
    arr_tif = copy.deepcopy(arr_tif0)
    [rows, cols] = arr_tif.shape 
    for i in range(rows - 1): 
        for j in range(cols-1): 
            if arr_tif[i, j] != -9999:
                if slope[i, j] != 0:
                    arr_tif[i, j] = (1-0.1*b[i, j]*(1+0.1*slope[i, j]**0.3))
                else:
                    arr_tif[i, j] = (1-0.1*b[i, j])
    return arr_tif
b_th1 = b_theta1(bv_arr_1, b, slope)
b_th2 = b_theta2(bv_arr_1, b, slope)

In [184]:
def b_theta3(arr_tif0, b, slope):
    arr_tif = copy.deepcopy(arr_tif0)
    [rows, cols] = arr_tif.shape 
    for i in range(rows - 1): 
        for j in range(cols-1): 
            if arr_tif[i, j] != -9999:
                if slope[i, j] != 0:
                    arr_tif[i, j] = (1-0.06*b[i, j]*(1+0.1*slope[i, j]**0.3))
                else:
                    arr_tif[i, j] = (1-0.06*b[i, j])
    return arr_tif
b_th3 = b_theta3(bv_arr_1, b, slope)

In [185]:
def b_theta4(arr_tif0, b, slope):
    arr_tif = copy.deepcopy(arr_tif0)
    [rows, cols] = arr_tif.shape 
    for i in range(rows - 1): 
        for j in range(cols-1): 
            if arr_tif[i, j] != -9999:
                if slope[i, j] != 0:
                    arr_tif[i, j] = (1-0.02*b[i, j]*(1+0.1*slope[i, j]**0.3))
                else:
                    arr_tif[i, j] = (1-0.02*b[i, j])
    return arr_tif
b_th4 = b_theta4(bv_arr_1, b, slope)

In [187]:
b_th1[555,336]

1.7119627088573688

In [188]:
b_th2[555,336]

0.9669198718589248

In [189]:
b_th3[555,336]

0.9801519231153548

### (3) 计算阻力

In [190]:
def rows_cols(ind, path_one):
    rows = []
    cols = []
    if ind < len(path_one) - 2:
        while ind < len(path_one) - 2:
            rows += path_one[ind+1][0]
            cols += path_one[ind+1][1]
            ind +=1
        return [rows,cols]
    else:
        print ('ind out of index of path_one')
def bv_path(all_paths,reg_arr0,bv_arr0,b_th):
    bv_arr = copy.deepcopy( bv_arr0)
    reg_arr = copy.deepcopy(reg_arr0)
    for i in range(0,len(path)-2):
        ind_0 = path[i]
        print(ind_0)
        print(reg_arr[ind_0][0])
        if reg_arr[ind_0][0] ==0:
            ind_1 = rows_cols(i, path)
            print(ind_1)
            bv_arr[ind_0] = b_th[ind_1].prod()
            reg_arr[ind_0] = 1
        else:
            break
    return bv_arr
def bv_paths(all_paths,reg_arr0,bv_arr0,b_th):
    bv_arr = copy.deepcopy( bv_arr0)
    reg_arr = copy.deepcopy(reg_arr0)
    for path in all_paths:
        for i in range(0,len(path)-2):
            ind_0 = path[i]
           # print(ind_0)
            #print(reg_arr[ind_0][0])
            if reg_arr[ind_0][0] ==0:
                ind_1 = rows_cols(i, path)
                #print(ind_1)
                bv_arr[ind_0] = b_th[ind_1].prod()
                reg_arr[ind_0] = 1
            else:
                break
    return bv_arr

In [191]:
bv_result1 = bv_paths(river_s,reg_arr,bv_arr,b_th1) #
bv_result2 = bv_paths(river_s,reg_arr,bv_arr,b_th2) #

In [194]:
bv_result3 = bv_paths(river_s,reg_arr,bv_arr,b_th3) #

In [195]:
bv_result4 = bv_paths(river_s,reg_arr,bv_arr,b_th4) #

In [209]:
bv_result1[556,335]

1.7119627088573688

In [193]:
bv_result2[556,335]

1.0

In [74]:
bv_result3[556,335]

1.0

In [210]:
bv_result4[556,335]

0.9933839743717849

In [211]:
np.savetxt('ASCII/bv_result_1.txt',bv_result1,fmt='%0.6f') # 没有前6行
np.savetxt('ASCII/bv_result_2.txt',bv_result2,fmt='%0.6f') # 没有前6行
np.savetxt('ASCII/bv_result_3.txt',bv_result3,fmt='%0.6f') # 没有前6行
np.savetxt('ASCII/bv_result_4.txt',bv_result4,fmt='%0.6f') # 没有前6行

In [212]:
add_begin('bv_result_1')
add_begin('bv_result_2')
add_begin('bv_result_3')
add_begin('bv_result_4')

In [128]:
all_paths[0]

[[[5], [83]], [[6], [82]], [[7], [81]], [[7], [80]]]

In [127]:
rows_cols(0, all_paths[0])

[[6, 7], [82, 81]]

In [134]:
reg_arr[[6, 7], [82, 81]]

array([0.59206212, 0.60365505])

### test 4

In [53]:
import numpy as np

import matplotlib.pyplot as plt

x=np.linspace(-5,5,1000)  #这个表示在-5到5之间生成1000个x值

y1=[i**2 for i in x]  #对上述生成的1000个数循环用sigmoid公式求对应的y

plt.plot(x,y1)  #用上述生成的1000个xy值对生成1000个点

plt.show()  #绘制图像

## 6 动力乘阻力

In [5]:
#动力结果
pow_all_1 = np.loadtxt('ASCII/pow_all_1.txt',skiprows = 6)
pow_all_2 = np.loadtxt('ASCII/pow_all_2.txt',skiprows = 6)

In [4]:
#阻力结果
bv_result1 = np.loadtxt('ASCII/bv_result_1.txt',skiprows = 6)
bv_result2 = np.loadtxt('ASCII/bv_result_2.txt',skiprows = 6)
bv_result3 = np.loadtxt('ASCII/bv_result_3.txt',skiprows = 6)
bv_result4 = np.loadtxt('ASCII/bv_result_4.txt',skiprows = 6)

In [70]:
bv_result1[5,83]

2.09044101267082

In [71]:
bv_result2[5,83]

0.8617877177023352

In [47]:
def del_99(arr_tif0,bounds,null_d):
    '''初始化空值为null_d
    '''
    arr_tif = copy.deepcopy(arr_tif0)
    for i in bounds: 
        for j in i: 
            arr_tif[j] = [null_d]
    return arr_tif
#del_99(fd,bound_s,0)
def ini_tif(arr_tif0,d,jr,bounds,null_d):
    '''初始化值为d
    '''
    arr_tif  = copy.deepcopy(arr_tif0) 
    [rows, cols] = arr_tif.shape 
    for i in range(rows - 1): 
        for j in range(cols-1): 
            if arr_tif[i, j] != -9999:
                if jr[i, j] == 1:
                    arr_tif[i, j] = null_d
                else:
                    arr_tif[i, j] = d
    arr_tif1 = copy.deepcopy(arr_tif) 
    arr_tif2 = del_99(arr_tif1,bound_s,null_d)
    return arr_tif2

am_1 = ini_tif(fd,0,jr,bound_s,0)
am_2 = ini_tif(fd,0,jr,bound_s,0)
am_3 = ini_tif(fd,0,jr,bound_s,0)
am_4 = ini_tif(fd,0,jr,bound_s,0)

def mul(arr0,arr1,arr2,d):
    arr0 = copy.deepcopy(arr0)
    [rows, cols] = arr1.shape 
    for i in range(rows - 1): 
        for j in range(cols-1): 
            if arr1[i,j] != -9999:
                if arr1[i,j] != -9999 or arr2[i,j] != -9999:
                    arr0[i,j] = arr1[i,j]*arr2[i,j]
                else:
                    arr0[i,j] = d
    return arr0

In [49]:
am1 = mul(am_1,pow_all_1,bv_result1,0)

In [50]:
am2 = mul(am_2,pow_all_2,bv_result2,0)

In [51]:
am3 = mul(am_3,pow_all_2,bv_result3,0)

In [52]:
am4 = mul(am_4,pow_all_2,bv_result4,0)

In [53]:
am1[555,336]

3.934019

In [54]:
am2[555,336]

2.306095

In [55]:
np.savetxt('ASCII/am1.txt',am1,fmt='%0.6f') # 没有前6行
np.savetxt('ASCII/am2.txt',am2,fmt='%0.6f') # 没有前6行

In [56]:
np.savetxt('ASCII/am3.txt',am3,fmt='%0.6f') # 没有前6行

In [57]:
np.savetxt('ASCII/am4.txt',am4,fmt='%0.6f') # 没有前6行

In [58]:
np.savetxt('ASCII/am1.txt',am1,fmt='%0.6f') # 没有前6行
def add_begin(name):
    with open('ASCII/begin.txt', 'r+') as f:
        content = f.read() 
        with open('ASCII/{}.txt'.format(name), 'r+') as f:
            content0 = f.read() 
            f.seek(0, 0)
            f.write(content + '\n' + content0)
            f.close()
        f.close()
add_begin('am1')

In [59]:
add_begin('am1')
add_begin('am2')

In [60]:
add_begin('am3')

In [61]:
add_begin('am4')

### test5 array

In [4]:
a1=np.array([[1,2,3],[4,5,6],[7,8,9]])

In [5]:
a1

array([[1, 2, 3],
       [4, 5, 6],
       [7, 8, 9]])

In [32]:
a1*a1

array([[ 1,  4,  9],
       [16, 25, 36],
       [49, 64, 81]])

### test6 追加文本

In [8]:
np.savetxt('ASCII/am.txt',a1,fmt='%0.6f') # 没有前6行

In [9]:
with open('ASCII/am.txt', 'r+') as f:
    content = f.read() 
    print(content)
    f.seek(0, 0)
    f.write('writer:Fatsheep\n'+content)
    f.close()

1.000000 2.000000 3.000000
4.000000 5.000000 6.000000
7.000000 8.000000 9.000000



In [53]:
def add_begin(name):
    with open('ASCII/begin.txt', 'r+') as f:
        content = f.read() 
        with open('ASCII/{}.txt'.format(name), 'r+') as f:
            content0 = f.read() 
            f.seek(0, 0)
            f.write(content + '\n' + content0)
            f.close()
        f.close()

In [56]:
add_begin('am')